## Boston Gender with Python!!! and machine learning!!!


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import csv
import numpy as np
import random
import scipy.stats as ss
import matplotlib.pyplot as plt



In [2]:
#from https://pypi.python.org/pypi/gender-guesser/0.4.0
#uses python 3 

import gender_guesser.detector as gender
d = gender.Detector()
print(d.get_gender(u"Bob"))

print(d.get_gender(u"Sally"))

print(d.get_gender(u"Pauley")) # should be androgynous


male
female
andy


In [3]:
#import the csv i downloaded from the website with the names -- could robotize this later..
# they have an API https://data.cityofboston.gov/Finance/Employee-Earnings-Report-2015/ah28-sywy
pay_2016 = pd.read_csv("Employee_Earnings_Report_2016.csv")

#pay_2015 = pd.read_csv("Employee_Earnings_Report_2015.csv")
#pay_2014 = pd.read_csv("Employee_Earnings_Report_2014.csv")
pay_2016.head()

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN/EDUCATION INCENTIVE,TOTAL EARNINGS,POSTAL
0,"Abadi,Kidani A",Assessing Department,Property Officer,$46291.98,NaN,$300.00,NaN,NaN,NaN,NaN,$46591.98,02118
1,"Abasciano,Joseph",Boston Police Department,Police Officer,$6933.66,NaN,$850.00,$205.92,$74331.86,NaN,$15258.44,$97579.88,02132
2,"Abban,Christopher John",Boston Fire Department,Fire Fighter,$103442.22,NaN,$550.00,$15884.53,NaN,$4746.50,NaN,$124623.25,02132
3,"Abbasi,Sophia",Green Academy,Manager (C) (non-ac),$18249.83,NaN,NaN,NaN,NaN,NaN,NaN,$18249.83,02148
4,"Abbate-Vaughn,Jorgelina",BPS Ellis Elementary,Teacher,$84410.28,NaN,$1250.00,NaN,NaN,NaN,NaN,$85660.28,02481


In [4]:
pay_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22046 entries, 0 to 22045
Data columns (total 12 columns):
NAME                         22046 non-null object
DEPARTMENT_NAME              22046 non-null object
TITLE                        22046 non-null object
REGULAR                      21463 non-null object
RETRO                        416 non-null object
OTHER                        12845 non-null object
OVERTIME                     6690 non-null object
INJURED                      1378 non-null object
DETAIL                       2425 non-null object
QUINN/EDUCATION INCENTIVE    1290 non-null object
TOTAL EARNINGS               22046 non-null object
POSTAL                       22045 non-null object
dtypes: object(12)
memory usage: 2.0+ MB


what would I like to do now? for the graphic, I need to genderize, and then pick the top 100 people by pay.
the genderizer uses first name,so I need to make a column of first names..Ill start with the 2016 data set and aim to have this done soon to update my graphic...
...Ill have to experiment a bit with the genderizer so will pull out 20 names to test them..

In [5]:
d_2016 = pay_2016 #copying the dataset to avoid changes
#d_2015 = pay_2015
#d_2014 = pay_2014


#d_2016['regular'] = (d_2016['REGULAR'].replace( '[\$,)]','', regex=True ).astype(float))
#d_2016 = d_2016.rename(columns = {
#'REGULAR':"regular",
#"RETRO": "retro",})
                       
                       
d_2016.head()

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN/EDUCATION INCENTIVE,TOTAL EARNINGS,POSTAL
0,"Abadi,Kidani A",Assessing Department,Property Officer,$46291.98,NaN,$300.00,NaN,NaN,NaN,NaN,$46591.98,02118
1,"Abasciano,Joseph",Boston Police Department,Police Officer,$6933.66,NaN,$850.00,$205.92,$74331.86,NaN,$15258.44,$97579.88,02132
2,"Abban,Christopher John",Boston Fire Department,Fire Fighter,$103442.22,NaN,$550.00,$15884.53,NaN,$4746.50,NaN,$124623.25,02132
3,"Abbasi,Sophia",Green Academy,Manager (C) (non-ac),$18249.83,NaN,NaN,NaN,NaN,NaN,NaN,$18249.83,02148
4,"Abbate-Vaughn,Jorgelina",BPS Ellis Elementary,Teacher,$84410.28,NaN,$1250.00,NaN,NaN,NaN,NaN,$85660.28,02481


In [6]:
#do I need to make sure other stuff is string?

d_2016['REGULAR'] = (d_2016['REGULAR'].replace( '[\$,)]','', regex=True ).astype(float)).round()
d_2016['RETRO'] = (d_2016['RETRO'].replace( '[\$,)]','', regex=True ).astype(float)).round()
d_2016['OTHER'] = (d_2016['OTHER'].replace( '[\$,)]','', regex=True ).astype(float)).round()
d_2016['OVERTIME'] = (d_2016['OVERTIME'].replace( '[\$,)]','', regex=True ).astype(float)).round()
d_2016['INJURED'] = (d_2016['INJURED'].replace( '[\$,)]','', regex=True ).astype(float)).round()
d_2016['DETAIL'] = (d_2016['DETAIL'].replace( '[\$,)]','', regex=True ).astype(float)).round()
d_2016['QUINN/EDUCATION INCENTIVE'] = (d_2016['QUINN/EDUCATION INCENTIVE'].replace( '[\$,)]','', regex=True ).astype(float)).round()
d_2016['TOTAL EARNINGS'] = (d_2016['TOTAL EARNINGS'].replace( '[\$,)]','', regex=True ).astype(float)).round()


d_2016.head()

d_2016 = d_2016.rename(columns = {
'REGULAR':"salary",
'RETRO':"retroactive",
'OTHER':'other',
'OVERTIME':'overtime',
'DETAIL':'detail',
'INJURED':'injured',
'QUINN/EDUCATION INCENTIVE':'quinn_education_incentive',
'TOTAL EARNINGS':'totalSalary',
'NAME':'name',
"DEPARTMENT_NAME" : "dept",
"TITLE" : "title",
"POSTAL" : "zip_code",
    
    
})
d_2016.head()


#the csv used for d3(v3) jsI want to update has this: columns are name, 
#firstName, dept, salary, totalSalary(still has $), gender -- is just F or M 



,name,dept,title,salary,retroactive,other,overtime,injured,detail,quinn_education_incentive,totalSalary,zip_code
0,"Abadi,Kidani A",Assessing Department,Property Officer,46292.0,NaN,300.0,NaN,NaN,NaN,NaN,46592.0,02118
1,"Abasciano,Joseph",Boston Police Department,Police Officer,6934.0,NaN,850.0,206.0,74332.0,NaN,15258.0,97580.0,02132
2,"Abban,Christopher John",Boston Fire Department,Fire Fighter,103442.0,NaN,550.0,15885.0,NaN,4746.0,NaN,124623.0,02132
3,"Abbasi,Sophia",Green Academy,Manager (C) (non-ac),18250.0,NaN,NaN,NaN,NaN,NaN,NaN,18250.0,02148
4,"Abbate-Vaughn,Jorgelina",BPS Ellis Elementary,Teacher,84410.0,NaN,1250.0,NaN,NaN,NaN,NaN,85660.0,02481


Round salaries to nearest dollar


In [7]:
import math
h = int(round(4.2))
h

4

### I ll split on the names
and now can see that there are some middle names 
so I'm going to use Regex to clean a bit..
if there is a Capitalized letter with nothing after, or a capitalized letter with a period, I want to split it, but otherwise Ill keep the two names, don't know how genderizer will handle them but well see..


had to do the splitting off of first name in two steps that probably should be one so can review later..below might help

#see this
#https://stackoverflow.com/questions/29585106/pandas-how-to-split-twice-a-given-field
 #try this:
trimmed_list = [i.replace('.', ',').split(',') for i in sbj[0].tolist() if i != ""]
item_list = [item.strip(whitespace) for item in itertools.chain(*trimmed_list)]
item_count = Counter(item_list)


In [8]:
firstName = d_2016.name.apply(lambda x: x.split(',')[1])
y = [name.split(" ")[0] for name in firstName]
y
#[]f_name.apply(lambda x: x.split(',')[1])
#first_name

#name_split = [name.split(',') for name in names]
#newlist = []
#for name in name_split:
 #   first = name[1]
  #  newlist.append(first)
#y = []
#for name in f_name:
 #   x = name.split(" ")
  #  y.append(x[0])
#print(y)




['Kidani',
 'Joseph',
 'Christopher',
 'Sophia',
 'Jorgelina',
 'James',
 'Erin',
 'John',
 'Angela',
 'Donna',
 'Amira',
 'Musiri',
 'Naima',
 'Lisa',
 'Maha',
 'Ramadani',
 'Theodora',
 'Jean',
 'Eliphete',
 'Marc',
 'Linda',
 'Adedolapo',
 'Obaida',
 'Jocelyne',
 'Caroline',
 'Randolph',
 'Sharon',
 'Rafael',
 'Patrick',
 'Daniel',
 'Harriet',
 'Sarah',
 'Cesar',
 'Iris',
 'Julie',
 'Moises',
 'Wayne',
 'Farouqua',
 'Patricia',
 'Jean',
 'Marie',
 'AnnMarie',
 'William',
 'Richard',
 'Blanca',
 'Danitza',
 'Dora',
 'Jasmine',
 'Johanna',
 'Rafael',
 'Ricardo',
 'Terisa',
 'Ilana',
 'Lodz',
 'Geoffrey',
 'Lyn',
 'Jean',
 'Christopher',
 'Ariel',
 'Carina',
 'Jose',
 'Tanya',
 'Robert',
 'Jessica',
 'S.Farhan',
 'Donna',
 'Nicole',
 'Carey',
 'Christopher',
 'Christopher',
 'Clyde',
 'Daniel',
 'Daune',
 'Duel',
 'Eleanor',
 'Elizabeth',
 'Frederick',
 'Gillian',
 'Heather',
 'Jasenta',
 'John',
 'Kwame',
 'Lucille',
 'Natasha',
 'Rebecca',
 'Terrence',
 'Leslie',
 'Aileen',
 'Anthony

create a column called first name and one called gender

In [9]:
import gender_guesser.detector as gender
d = gender.Detector()

gender = []

for name in y:
    gender_it = d.get_gender(name)
    gender.append(gender_it)

d_2016["firstName"] = y
d_2016["gender"] = gender

Do the whole data set, create a column of first name and a column of gender, i need to make it match eventually
what I used in my D3 csv to avoid hassle when recreating the graphic

In [10]:

d_2016.head()

,name,dept,title,salary,retroactive,other,overtime,injured,detail,quinn_education_incentive,totalSalary,zip_code,firstName,gender
0,"Abadi,Kidani A",Assessing Department,Property Officer,46292.0,NaN,300.0,NaN,NaN,NaN,NaN,46592.0,02118,Kidani,unknown
1,"Abasciano,Joseph",Boston Police Department,Police Officer,6934.0,NaN,850.0,206.0,74332.0,NaN,15258.0,97580.0,02132,Joseph,male
2,"Abban,Christopher John",Boston Fire Department,Fire Fighter,103442.0,NaN,550.0,15885.0,NaN,4746.0,NaN,124623.0,02132,Christopher,male
3,"Abbasi,Sophia",Green Academy,Manager (C) (non-ac),18250.0,NaN,NaN,NaN,NaN,NaN,NaN,18250.0,02148,Sophia,female
4,"Abbate-Vaughn,Jorgelina",BPS Ellis Elementary,Teacher,84410.0,NaN,1250.0,NaN,NaN,NaN,NaN,85660.0,02481,Jorgelina,unknown


change the order of the columns


In [11]:
cols = list(d_2016.columns.values)
cols

['name',
 'dept',
 'title',
 'salary',
 'retroactive',
 'other',
 'overtime',
 'injured',
 'detail',
 'quinn_education_incentive',
 'totalSalary',
 'zip_code',
 'firstName',
 'gender']

In [12]:
d_2016 = d_2016[[
 'name',
    'firstName',
 'gender',
 'salary',
    'totalSalary',
'dept',
 'title',
 'retroactive',
 'other',
 'overtime',
 'injured',
 'detail',
 'quinn_education_incentive',
 'zip_code', 
 ]]
d_2016.head()

,name,firstName,gender,salary,totalSalary,dept,title,retroactive,other,overtime,injured,detail,quinn_education_incentive,zip_code
0,"Abadi,Kidani A",Kidani,unknown,46292.0,46592.0,Assessing Department,Property Officer,NaN,300.0,NaN,NaN,NaN,NaN,02118
1,"Abasciano,Joseph",Joseph,male,6934.0,97580.0,Boston Police Department,Police Officer,NaN,850.0,206.0,74332.0,NaN,15258.0,02132
2,"Abban,Christopher John",Christopher,male,103442.0,124623.0,Boston Fire Department,Fire Fighter,NaN,550.0,15885.0,NaN,4746.0,NaN,02132
3,"Abbasi,Sophia",Sophia,female,18250.0,18250.0,Green Academy,Manager (C) (non-ac),NaN,NaN,NaN,NaN,NaN,NaN,02148
4,"Abbate-Vaughn,Jorgelina",Jorgelina,unknown,84410.0,85660.0,BPS Ellis Elementary,Teacher,NaN,1250.0,NaN,NaN,NaN,NaN,02481


messing about with the name split 

name_split = [name.split(',') for name in names]
newlist = []
for name in name_split:
    first = name[1]
    newlist.append(first)
y = []
for name in newlist:
    x = name.split(" ")
    y.append(x[0])

#add y as a column in the data set

d_2016["first_names"] = y
d_2016.head()


messing about

import gender_guesser.detector as gender
d = gender.Detector()

gender = []

for name in y:
    gender_it = d.get_gender(name)
    gender.append(gender_it)
d_2016["gender"] = gender
d_2016.head()

shortdf = d_2016[["NAME","first_names","gender","REGULAR"]] #I can add stuff here later and rerun for my csv, ie department
print(shortdf.sort_values(by=['gender'],ascending=True))

now I'd like to see how many there are of each type in the new gender column...

In [13]:
d_2016['gender'].value_counts() #andy is androgynous

female           9206
male             9097
unknown          2210
mostly_female     985
mostly_male       376
andy              172
Name: gender, dtype: int64

df.loc[df['First Season'] > 1990, 'First Season'] = 1

In [14]:
d_2016.loc[d_2016['gender'] == "mostly_female", "gender"] = "female"
d_2016.loc[d_2016['gender'] == "mostly_male", "gender"] = "male"
d_2016['gender'].value_counts() #andy is androgynous

female     10191
male        9473
unknown     2210
andy         172
Name: gender, dtype: int64

I'm going to group mostly's into their respective groups

im going to sort by top 100 earners to see how many of those have different value counts since
that's what I use in my chart


these two women are listed as unknowns, since I looked them up im going to fix in the data set..
13129          McCreary,Makeeba   Makeeba                 Communications   
6050   Edouard-Vincent,Marice M    Marice  Asst Superintendent-Network A 

In [15]:
d_2016.loc[d_2016['name'] == "McCreary,Makeeba", "gender"] = "female"
d_2016.loc[d_2016['name'] == "Edouard-Vincent,Marice M", "gender"] = "female"
d_2016['gender'].value_counts()

female     10193
male        9473
unknown     2208
andy         172
Name: gender, dtype: int64

In [16]:
d_2016.head()

,name,firstName,gender,salary,totalSalary,dept,title,retroactive,other,overtime,injured,detail,quinn_education_incentive,zip_code
0,"Abadi,Kidani A",Kidani,unknown,46292.0,46592.0,Assessing Department,Property Officer,NaN,300.0,NaN,NaN,NaN,NaN,02118
1,"Abasciano,Joseph",Joseph,male,6934.0,97580.0,Boston Police Department,Police Officer,NaN,850.0,206.0,74332.0,NaN,15258.0,02132
2,"Abban,Christopher John",Christopher,male,103442.0,124623.0,Boston Fire Department,Fire Fighter,NaN,550.0,15885.0,NaN,4746.0,NaN,02132
3,"Abbasi,Sophia",Sophia,female,18250.0,18250.0,Green Academy,Manager (C) (non-ac),NaN,NaN,NaN,NaN,NaN,NaN,02148
4,"Abbate-Vaughn,Jorgelina",Jorgelina,unknown,84410.0,85660.0,BPS Ellis Elementary,Teacher,NaN,1250.0,NaN,NaN,NaN,NaN,02481


In [17]:
salarySorted = d_2016.sort_values(by='salary', ascending=False)
salarySorted.head()



,name,firstName,gender,salary,totalSalary,dept,title,retroactive,other,overtime,injured,detail,quinn_education_incentive,zip_code
3281,"Chang,Tommy",Tommy,male,259484.0,271734.0,Superintendent,Superintendent,NaN,12250.0,NaN,NaN,NaN,NaN,02131
6323,"Evans,William B",William,male,221323.0,245305.0,Boston Police Department,Commissioner (Bpd),8598.0,15384.0,NaN,NaN,NaN,NaN,02127
20960,"Walsh,John Francis",John,male,207578.0,231707.0,Boston Fire Department,Chief of Support Services,NaN,19421.0,4708.0,NaN,NaN,NaN,02184
7000,"Fontana,Gerard T.",Gerard,male,206823.0,226244.0,Boston Fire Department,Chief of Field Services,NaN,19421.0,NaN,NaN,NaN,NaN,02169
8095,"Granara,David Richard",David,male,203136.0,215264.0,Boston Fire Department,DepFireChiefAdmn-AdvTechnician,NaN,12127.0,NaN,NaN,NaN,NaN,02128


so i have what i need above to export to csv and update my graph...START HERE
-- after updating, play with bokeh here? do a new sheet to update FBI crime project?

the csv used for d3(v3) jsI want to update has this: columns are name, firstName, dept, salary, totalSalary(still has $), gender -- is just F or M 
so make a truncated version of the csv
name in this style to be like existing csv:
a_data_boston_2014_employee_top_100

In [18]:
#make csv with just the earlier data columns ie 6 of them newdf = df[(df['column_one']>2004) & (df['column_two']==9)]
#AND JUST PICK THE TOP 100 
shortForCsv = salarySorted[["name", "firstName","dept", "salary", "totalSalary","gender"]].head(100)
#df1 = df[['a','b']]
#df1 = df.ix[:,0:2]
shortForCsv['gender'].value_counts()



male      85
female    15
Name: gender, dtype: int64

In [19]:
shortForCsv.head()

,name,firstName,dept,salary,totalSalary,gender
3281,"Chang,Tommy",Tommy,Superintendent,259484.0,271734.0,male
6323,"Evans,William B",William,Boston Police Department,221323.0,245305.0,male
20960,"Walsh,John Francis",John,Boston Fire Department,207578.0,231707.0,male
7000,"Fontana,Gerard T.",Gerard,Boston Fire Department,206823.0,226244.0,male
8095,"Granara,David Richard",David,Boston Fire Department,203136.0,215264.0,male


bring the csv formula lower later..

I havent done anything about the gender groups yet, I need to make them into M or F and get rid of or 
check the unknowns and mostly's...'

In [20]:
shortForCsv['gender'].value_counts()

male      85
female    15
Name: gender, dtype: int64

In [21]:
unknowns = shortForCsv[(shortForCsv['gender']== "unknown")]
#MM = shortForCsv[(shortForCsv['gender']== "mostly_male")]
#MF = shortForCsv[(shortForCsv['gender']== "mostly_female")]
print(unknowns) 
#since its a fairly short list Im going to look them up on the intraweb but this is something I should automate or skip those 
#names, or make my own additional filter using social security names?
#i Can also add the ones Ive checked to these lists so they are 

Empty DataFrame
Columns: [name, firstName, dept, salary, totalSalary, gender]
Index: []


McCreary - F
https://www.bostonpublicschools.org/site/default.aspx?PageType=3&ModuleInstanceID=9715&ViewID=7b97f7ed-8e5e-4120-848f-a8b4987d588f&RenderLoc=0&FlexDataID=7541&PageID=4853

 Edouard-Vincent,Marice - F
https://twitter.com/netsuptb?lang=en

McGovern,Ryan J -- M
http://www.firehouse.com/news/11252963/after-falling-through-floor-boston-firefighter-thankful-for-minor-injuries

Franklin-Hodge,Jascha -- M
https://www.boston.gov/departments/innovation-and-technology/jascha-franklin-hodge

  Nee,Kelly A  -- F
https://www.youtube.com/watch?v=2x7OrSKi_04

 Driscoll,Mary E      Mary  Asst Superintendent-Network A - F
 https://www.bc.edu/schools/csom/lynchacademy/alumni/2012/Driscoll.html
 
  Hung,Kelly M     Kelly  Asst Superintendent-Network A  153503.93 -- F
  
  https://www.bc.edu/schools/csom/lynchacademy/alumni/2014/KHung.html

NEED TO FIX THE WEB LINK ON THE CHARTS PAGE TO GO TO 2016

I could look up the mostly and unknowns to confirm their gender -- its just 7 peeps

In [22]:
shortForCsv.head()

,name,firstName,dept,salary,totalSalary,gender
3281,"Chang,Tommy",Tommy,Superintendent,259484.0,271734.0,male
6323,"Evans,William B",William,Boston Police Department,221323.0,245305.0,male
20960,"Walsh,John Francis",John,Boston Fire Department,207578.0,231707.0,male
7000,"Fontana,Gerard T.",Gerard,Boston Fire Department,206823.0,226244.0,male
8095,"Granara,David Richard",David,Boston Fire Department,203136.0,215264.0,male


it would be nice to add additioal charts to that page, like for 2016 are the highest paid workers in 
each dept male or female? not sure how to make a simple graph like that, dot matrix?

it seems like tommy chang is top earner but comes up as superintendent, rather than school system. going to take a look at how they label departments in the larger data set..

In [23]:
d_2016['dept'].value_counts() 
shortForCsv["dept"].value_counts() 

Boston Fire Department            57
Boston Police Department          21
Asst Superintendent-Network A      7
Mayor's Office                     2
Chief Operating Officer            2
Achievement Gap                    1
BPS Human Resource Team            1
Strategy Department                1
Fam & Student Engagemt             1
Boston Public Library              1
Kennedy, EM Health Academy         1
Dpt of Innovation & Technology     1
Communications                     1
Law Department                     1
BPS Special Education              1
Superintendent                     1
Name: dept, dtype: int64

I'm going to group the schools for the top 100 under Boston Public Schools

In [24]:
test = shortForCsv[(shortForCsv['dept']== "Fam & Student Engagemt")]
test1 = shortForCsv[(shortForCsv['dept']== "Achievement Gap")]
test2 = shortForCsv[(shortForCsv['dept']== "Kennedy, EM Health Academy")]

test2


,name,firstName,dept,salary,totalSalary,gender
20879,"Walker Gregory,Caren S",Caren,"Kennedy, EM Health Academy",157477.0,159977.0,female


In [27]:
shortForCsv.loc[shortForCsv['dept'] == "Asst Superintendent-Network A", "dept"] = "Boston Public Schools"
shortForCsv.loc[shortForCsv['dept'] == "BPS Special Education", "dept"] = "Boston Public Schools"
shortForCsv.loc[shortForCsv['dept'] == "Fam & Student Engagemt", "dept"] = "Boston Public Schools"
shortForCsv.loc[shortForCsv['dept'] == "Achievement Gap", "dept"] = "Boston Public Schools"
shortForCsv.loc[shortForCsv['dept'] == "Kennedy, EM Health Academy", "dept"] = "Boston Public Schools"
shortForCsv.loc[shortForCsv['dept'] == "BPS Human Resource Team", "dept"] = "Boston Public Schools"
shortForCsv.loc[shortForCsv['dept'] == "Superintendent", "dept"] = "Boston Public Schools"
shortForCsv.loc[shortForCsv['dept'] == "Law Department", "dept"] = "Law"
shortForCsv.loc[shortForCsv['dept'] == "Dpt of Innovation & Technology", "dept"] = "Innovation & Technology"


shortForCsv["dept"].value_counts()

Boston Fire Department      57
Boston Police Department    21
Boston Public Schools       13
Chief Operating Officer      2
Mayor's Office               2
Boston Public Library        1
Strategy Department          1
Communications               1
Law                          1
Innovation & Technology      1
Name: dept, dtype: int64

In [28]:
shortForCsv.to_csv("top100_2016", index=False)

In [44]:
b = d_2016[(d_2016['dept']== "Superintendent")]
b
#could check to see if all these people are in BPS to change the category overall..

,name,firstName,gender,salary,totalSalary,dept,title,retroactive,other,overtime,injured,detail,quinn_education_incentive,zip_code
3281,"Chang,Tommy",Tommy,male,259484.0,271734.0,Superintendent,Superintendent,NaN,12250.0,NaN,NaN,NaN,NaN,02131
3951,"Connolly,Barbara F",Barbara,female,114822.0,116889.0,Superintendent,Special Assistant (D),NaN,2066.0,NaN,NaN,NaN,NaN,02127
7660,"Giampietro,Erika",Erika,female,80628.0,82061.0,Superintendent,Special Assistant (E),NaN,1433.0,NaN,NaN,NaN,NaN,02116
9331,"Holland,Albert D",Albert,male,68320.0,68320.0,Superintendent,Employment Agreement,NaN,NaN,NaN,NaN,NaN,NaN,02122
9787,"Jackson,Queon N.",Queon,unknown,71002.0,100732.0,Superintendent,Asst Director (Basas 10B),NaN,29730.0,NaN,NaN,NaN,NaN,02186
13200,"McDonough,John P",John,male,4568.0,172999.0,Superintendent,Superintendent,NaN,168432.0,NaN,NaN,NaN,NaN,96753
16885,"Ramos,Lydia L",Lydia,female,69664.0,69664.0,Superintendent,Senior Advisor (E),NaN,NaN,NaN,NaN,NaN,NaN,02122


In [235]:
import seaborn as sns
#x = shortForCsv["dept"]
#y = shortForCsv["gender"]
sns.lmplot("dept", "gender", data=shortForCsv, fit_reg=False)
#ax = sns.stripplot(x=shortForCsv["dept"])

#d_2016["firstName"] = y
#d_2016["gender"] = gender

TypeError: Could not convert malemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalefemalemalemalemalemalemalemalemalefemalefemalemalemalemalemalemalemalefemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalemalefemalemalemalemalemalefemalemalemalemalemalefemalemalemalefemalemalemalemalemalefemalemalemalefemalemalemalemalemalemalemalemalefemalemalefemalemalefemalefemalemalefemale to numeric

Im going to add mostly female and mostly males to females and males respectively, and group androgynous and unknown into one group...maybe ill use them later 

#trying a second gender dectector
#couldn't geet it to work..
#import gender_guesser.detector as gender
import gender_detector.gender_detector as gender2
#detector = gender_detector('us') # It can also be ar, uk, uy.
gender2.guess('Marcos') # => 'male'
#from gender_detector import GenderDetector
#detector = GenderDetector('us') # It can also be ar, uk, uy.
#detector.guess('Marcos') # => 'male'

In [ ]:
max(pay_2016.regular)

In [ ]:
min(pay_2016.regular)

In [ ]:
x = pay_2016.iloc[:, 0:1]  #all rows, column 1

In [ ]:
x


In [ ]:
plt.plot(x)
plt.show()

In [ ]:
print x.mean(), x.max(), x.min(), x.mode()

In [ ]:
print birds["speed_2d"].mean(), birds["speed_2d"].max(), birds["speed_2d"].min(), birds["speed_2d"].mode()

We'll get started by simply plotting the latitude and longitude of a flight
trajectory for a single bird in a standard two dimensional plot.
Latitude and longitude are coordinates along the surface of a sphere,
where a standard 2-D plot is a plane
so we would expect very strong distortions
for the flight trajectories.
This is especially true because the trajectories
cover a large spatial region.
We're doing this however, primarily to have a quick way to look at the data
and develop more intuition about it.
We'll be doing an actual cartographic projection later in this case study.


ACCESS ONE COLUMN

In [ ]:
#so grab flight trajectories of single bird 
#birds[bird_name == "Eric"] WRONG! maybe works in np or somewhere?
#makes a boolean
ix = birds.bird_name == "Eric" #for pandas this is how to access column


In [ ]:
#one_bird = birds["latitude"]["longitude", "bird_name"]
#so hes using the ix seems to only grab values if they are True, ie Eric
x, y = birds.longitude[ix], birds.latitude[ix]

In [ ]:
birds.longitude[ix].head()

In [ ]:
plt.subplot(211)
plt.plot(x,y,"ob")
plt.subplot(212)
plt.plot(x,y,) #just shows default is prettier
plt.show()

plot more birds...


In [ ]:
birds.bird_name.tail(10)

using unique...

In [ ]:
#but he does something better than this
#ia = birds.bird_name == "Sanne"
#ic = birds.bird_name == "Nico"
names = pd.unique(birds.bird_name)
names

In [ ]:
#having trouble with this, see how he does it below
#he skips making a new dict or list and just puts them in the graph
#latlong = {}
#for i in names:
 #   ix = birds.bird_name == i
  #  x = birds.latitude[ix], 
   # y = birds.longitude[ix]
    #latlong[x] == y

#latlong

In [ ]:
#his example WOW! very cool...
#plt.figure(figsize=(7,7)) # he puts the figure creation outside the loop
#even though  i dont it so blocked it out
for name in names:
    ix = birds.bird_name == name
    x,y = birds.longitude[ix], birds.latitude[ix] #be aware the order you put them in, I had them opposite
    plt.plot(x, y, ".", label=name) #so for the legend to work, have to add the label here! otherwise it just says longitdue...guess the last label available to it
plt.xlabel("Longitude")
plt.ylabel("Latitude") #make sure labels match the data ie x to x and y to latitude
plt.legend(loc="lower right")
plt.show() #im putting this down here and imagine it will work?

In [ ]:
a,b = birds.latitude[ia], birds.longitude[ia]
c,d = birds.latitude[ic], birds.longitude[ic]

In [ ]:
plt.plot(x, y) #this also is quite different from the above, wonder why?
plt.plot(a, b)
plt.plot(c, d)
plt.show()

* Learn how to examine 2D flight speed of the birds
* Learn how to deal with data entries that are not numeric
* The data also contain estimates of two dimensional speeds of the birds.
That is how fast they were flying on a 2D
plane that is a local approximation to the curved surface of the earth.
Let's look at the speed data in a little bit more detail.

In [ ]:
ix = birds.bird_name == "Eric"
speed = birds.speed_2d[ix]
speed


hist doesnt like it when there are null values so the below
doesnt run..until they are eliminated..he starts by doing first 10 rows only, so good idea...

In [ ]:
plt.hist(speed[:10]) 
plt.show()

are there non number objects? ..below are useful tests using numpy

In [ ]:
np.isnan(speed) #i dont see any trues but this is a good way to look
np.isnan(speed).any()#this is much better since it tell us that there is a null value
np.sum(np.isnan(speed)) #sum all the isnans... I got alot more than him??

turn a list to its opposite with ~

In [ ]:
ind = np.isnan(speed)
ind #ind is a true and false list..the NANs are true
~ind #interesting way to get the mirror opposite of ind, the non nans are now TRUE

In [ ]:
#this looks the same as dropna which is less work! interesting to use that idea thought
ind = np.isnan(speed)
plt.hist(speed[~ind])
plt.show()

In [ ]:
#kind of weird looking though..?
# I used dropna here, 
u = speed.dropna()
plt.hist(u)
plt.show()

add x label, y label, and make the bins normalized


### it looks alot better adding more bins
* the y axis is normalized, meaning
that an integral over the histogram would be equal to 1.
* numpy.linspace(start, stop, num=50, endpoint=True, retstep=False, dtype=None)[source]
Return evenly spaced numbers over a specified interval.

In [ ]:
ind = np.isnan(speed)
plt.hist(speed[~ind], bins = np.linspace(0,30,20), normed = True)
#i see in docs normed has been deprecated so swapping that with "density",
#which doesnt work in python 2
plt.xlabel("2D Speed (m/s)")
plt.ylabel("Frequency")
plt.show()

### Pandas 
provide some planning functions as well,
although less customizable than PLT plots,
they readily deal with things like NaNs.
However you want to make sure that you always
examine your data carefully and look for the presence of NaNs
before preparing your plots.
Let's look at how we would make the histogram using pandas.

pandas uses the kind and range argument for kind and bin number...all of the nan issue happens under the hood

In [ ]:
birds.speed_2d.plot(kind='hist', range = [0,30]);
plt.xlabel("2D Speed");
plt.show()

## * Learn how deal with timestamped data using datetime
## * Learn how to measure elapsed time

year month and day, are all strings, we're going to  convert them into so-called daytime object, that
supports our arithmetic operations.

In [ ]:
birds.date_time[0:3] 

In [ ]:
import datetime

In [ ]:
datetime.datetime.today()

In [ ]:
time_1 = datetime.datetime.today()

In [ ]:
time_2 = datetime.datetime.today()
time_2

In [ ]:
time_2 - time_1 


the above is called a date time time delta object. he said they all end i 00 but mine doesnt, thogh the data set does

In [ ]:
date_str = birds.date_time[0]

In [ ]:
print date_str, type(date_str)

get rid of the last three characters

In [ ]:
date_str[:-3]

a long way to strip and prep this
this has two arguments, the string you are working on, then the format you want for that string -- it is picky though, if you don't peel off those last 3 digits it won't do it!

In [ ]:
datetime.datetime.strptime(date_str[:-3],"%Y-%m-%d %H:%M:%S")

the ooutput above is a datetime object, we can go over every row and create a new datetime object for each row

In [ ]:
#this was taking along time to run so might be messed up

timestamps = []
done = []
for date in birds.date_time:
    new_date = datetime.datetime.strptime(date_str[:-3],"%Y-%m-%d %H:%M:%S")
    new_time = str(new_date)
    timestamps.append(new_time)


In [ ]:
x = str(timestamps[0])

In [ ]:
x